In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap
# !{sys.executable} -m pip install pip rasterstats 

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-cities4forests'
# bucket_name = 'cities-urbanshift' 
aws_s3_dir = 'https://'+bucket_name+'.s3.eu-west-3.amazonaws.com/data'

In [ ]:
##Add Land use land cover dataset
WC = ee.ImageCollection("ESA/WorldCover/v100")
WorldCover = WC.first();
builtup = WorldCover.eq(50)

## define projection for use later
WCprojection = WC.first().projection();  
esaScale = WorldCover.projection().nominalScale();  

In [ ]:
## read Trees in Mosaic Landscapes tree cover dataset
TML = ee.ImageCollection('projects/wri-datalab/TML')
TreeCoverImg = TML.reduce(ee.Reducer.mean()).rename('b1')
TreeCovergt0 = TreeCoverImg.updateMask(TreeCoverImg.gt(0))

# # select only pixels with 10% or greater tree cover
# TreePctThreshold = 10 #whole numbers - 0-100
# TreeCover = TreeCover.updateMask(TreeCover.gte(TreePctThreshold))

# ## Reproject trees to match LULC projection
# TreeCover = TreeCover.reproject(crs= WCprojection)

# https://gis.stackexchange.com/questions/421422/google-earth-engine-image-no-data-mask
TreeDataMask = TreeCoverImg.unmask(-99).neq(-99)   # give no data pixels an abitrary value outside the range of the data. Then make a boolean raster using the assigned value. 

In [ ]:
## Create layer with tree cover in builtup areas
builtupTreeCover = TreeCovergt0.updateMask(builtup)

In [ ]:
# get list of c4f cities
boundary_georef = pd.read_csv(aws_s3_dir +'/boundaries/v_0/boundary_georef.csv')
# boundary_georef = pd.read_csv(aws_s3_dir +'/boundaries/AUE/boundary_georef_AUE.csv')

boundary_georef

# Compute indicator

In [ ]:
this_indicatorDF = pd.DataFrame() 

In [ ]:
# define calcuation function to get pixel counts, convert to percents and append to data frame
def CountCalcsDF(FC,DF):
    
    # reduce images to get vegetation and built-up pixel counts
    pixelcounts = builtupTreeCover.reduceRegions(FC,ee.Reducer.count().setOutputs(['TreeBuiltPixels']),esaScale) # larger scale (50+) required for large cities to avoid EE memory issues
    pixelcounts = builtup.reduceRegions(pixelcounts,ee.Reducer.count().setOutputs(['BuiltPixels']),esaScale) # larger scale (50+) required for large cities to avoid EE memory issues
    pixelcounts = TreeDataMask.reduceRegions(pixelcounts,ee.Reducer.anyNonZero().setOutputs(['TreeDataAvailable']),esaScale)
    pixelcounts = TreeCoverImg.reduceRegions(pixelcounts,ee.Reducer.mean().setOutputs(['TreeCoverMean']),esaScale)

    # convert pixel counts to area percentages and saves to FC as property
    def toPct(feat):
        BuiltpctEq = ee.Number(1).subtract((feat.getNumber('TreeBuiltPixels')).divide(feat.getNumber('BuiltPixels')))
        Builtpct = ee.Algorithms.If(feat.getNumber('TreeDataAvailable').eq(0),"NA",BuiltpctEq)
        Treepct = ee.Algorithms.If(feat.getNumber('TreeDataAvailable').eq(0),"NA",(feat.getNumber('TreeCoverMean').multiply(0.01)))
        return feat.set({
            'PctBuiltwoTree': Builtpct,
            'PctTreeCover': Treepct
      })

    pixelcounts = pixelcounts.map(toPct).select(['geo_id','PctBuiltwoTree','PctTreeCover'])

    # store in df and append
    df = geemap.ee_to_pandas(pixelcounts)
    df = df.rename(columns={'PctBuiltwoTree': 'GRE_1_4_percentBuiltupWithoutTreeCover'})
    DF = DF.append(df)
    return DF


In [ ]:
for i in range(0,1): #,len(boundary_georef)):
# for i in list(range(0, 56)) + list(range(57,120)) + list(range(121,131)) + list(range(132,len(boundary_georef))):
# for i in list(range(111,120)) + list(range(121,131)) + list(range(132,len(boundary_georef))):

    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    # process aoi level ------
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = ee.FeatureCollection(boundary_geo)
    
    this_indicatorDF = CountCalcsDF(boundary_geo_ee, this_indicatorDF)

    
    # process unit of analysis level ------
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    # boundary_geo_ee = ee.FeatureCollection(boundary_geo)
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    this_indicatorDF = CountCalcsDF(boundary_geo_ee, this_indicatorDF)


In [ ]:
this_indicatorDF#.tail(50)

# Workaround for timeout problems for specific geographies

In [ ]:
# if timeout problems for a ingesting a geography ("EEException: Request payload size exceeds the limit") or computing indicator ("Exception: Computation timed out"), save boundary as ee.FeatureCollection as EE asset in EE Code Editor then reference assets directly, one at a time.

# boundary_geo_ee = ee.FeatureCollection('users/emackres/AUE/boundary-USA-New_York-AUEt3')
# boundary_geo_ee = ee.FeatureCollection('users/emackres/AUE/boundary-CHN-Guangzhou_Guangdong-AUEt3')
# boundary_geo_ee = ee.FeatureCollection('users/emackres/AUE/boundary-FRA-Paris-AUEt3')
# boundary_geo_ee = ee.FeatureCollection('users/emackres/AUE/boundary-USA-Chicago-AUEt3')

# this_indicatorDF = CountCalcsDF(boundary_geo_ee, this_indicatorDF)

In [ ]:
this_indicatorDF#.tail(50)

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_v2test.csv') 
# cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_AUE.csv') 
cities_indicators

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    else:
        print("add new indicators")
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = this_indicatorDF,
                                            indicator_name = "GRE_1_4_percentBuiltupWithoutTreeCover")

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators_merged,
                                            new_indicator_table = this_indicatorDF,
                                            indicator_name = 'PctTreeCover')

In [ ]:
cities_indicators_merged

# Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
# aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = 'data/indicators/cities_indicators_v2test.csv'
# key_data = 'data/indicators/cities_indicators_AUE.csv'

cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')